# Dell PowerScale 文档加载器

[Dell PowerScale](https://www.dell.com/en-us/shop/powerscale-family/sf/powerscale) 是一个企业级横向扩展存储系统，它托管行业领先的 OneFS 文件系统，可以部署在本地或云端。

此文档加载器利用 PowerScale 的独特功能，可以确定自应用程序上次运行以来哪些文件已修改，并且仅返回修改过的文件以供处理。这将消除重新处理（分块和嵌入）未更改文件的需要，从而改进整体数据摄取工作流程。

此加载器要求启用 PowerScale 的 MetadataIQ 功能。更多信息可在我们的 GitHub 仓库中找到：[https://github.com/dell/powerscale-rag-connector](https://github.com/dell/powerscale-rag-connector)

## 概述
### 集成详情

| Class | Package | Local | Serializable | [JS support](https://js.langchain.com/docs/integrations/document_loaders/web_loaders/__module_name___loader)|
| :--- | :--- | :---: | :---: |  :---: |
| [PowerScaleDocumentLoader](https://github.com/dell/powerscale-rag-connector/blob/main/src/powerscale_rag_connector/PowerScaleDocumentLoader.py) | [powerscale-rag-connector](https://github.com/dell/powerscale-rag-connector) | ✅ | ❌ | ❌ | 
| [PowerScaleUnstructuredLoader](https://github.com/dell/powerscale-rag-connector/blob/main/src/powerscale_rag_connector/PowerScaleUnstructuredLoader.py) | [powerscale-rag-connector](https://github.com/dell/powerscale-rag-connector) | ✅ | ❌ | ❌ | 
### 加载器特性
| Source | Document Lazy Loading | Native Async Support
| :---: | :---: | :---: | 
| PowerScaleDocumentLoader | ✅ | ✅ | 
| PowerScaleUnstructuredLoader | ✅ | ✅ | 

## 设置

此文档加载器要求使用启用了 MetadataIQ 的 Dell PowerScale 系统。更多信息可在我们的 github 页面找到：[https://github.com/dell/powerscale-rag-connector](https://github.com/dell/powerscale-rag-connector)

### 安装

文档加载器存在于一个外部 pip 包中，可以使用标准工具进行安装。

In [ ]:
%pip install --upgrade --quiet  powerscale-rag-connector

## 初始化

现在我们可以实例化文档加载器：

### 通用文档加载器

我们的通用文档加载器可以按以下方式用于逐步加载 PowerScale 中的所有文件：

In [ ]:
from powerscale_rag_connector import PowerScaleDocumentLoader

loader = PowerScaleDocumentLoader(
    es_host_url="http://elasticsearch:9200",
    es_index_name="metadataiq",
    es_api_key="your-api-key",
    folder_path="/ifs/data",
)

### 非结构化加载器 Loader

`PowerScaleUnstructuredLoader` 可选地用于定位已更改的文件，以及自动处理文件，生成源文件的元素。这是通过 LangChain 的 `UnstructuredLoader` 类完成的。

In [ ]:
from powerscale_rag_connector import PowerScaleUnstructuredLoader

# Or load files with the Unstructured Loader
loader = PowerScaleUnstructuredLoader(
    es_host_url="http://elasticsearch:9200",
    es_index_name="metadataiq",
    es_api_key="your-api-key",
    folder_path="/ifs/data",
    # 'elements' mode splits the document into more granular chunks
    # Use 'single' mode if you want the entire document as a single chunk
    mode="elements",
)

字段：
- `es_host_url` 是 MetadataIQ Elasticsearch 数据库的端点
- `es_index_index` 是 PowerScale 写入其文件系统元数据的索引名称
- `es_api_key` 是您的 Elasticsearch API 密钥的 **编码** 版本
- `folder_path` 是要查询更改的 PowerScale 上的路径

## 加载

内部而言，PowerScale 和 MetadataIQ 的所有代码都是异步的，load 和 lazy load 方法将返回一个 Python 生成器。我们推荐使用 lazy load 函数。

In [ ]:
for doc in loader.load():
    print(doc)

[Document(page_content='' metadata={'source': '/ifs/pdfs/1994-Graph.Theoretic.Obstacles.to.Perfect.Hashing.TR0257.pdf', 'snapshot': 20834, 'change_types': ['ENTRY_ADDED']}),
Document(page_content='' metadata={'source': '/ifs/pdfs/New.sendfile-FreeBSD.20.Feb.2015.pdf', 'snapshot': 20920, 'change_types': ['ENTRY_MODIFIED']}),
Document(page_content='' metadata={'source': '/ifs/pdfs/FAST-Fast.Architecture.Sensitive.Tree.Search.on.Modern.CPUs.and.GPUs-Slides.pdf', 'snapshot': 20924, 'change_types': ['ENTRY_ADDED']})]


### 返回对象

两个文档加载器都会跟踪先前返回给您应用程序的文件。再次调用文档加载器时，它将仅返回自上次运行以来新增或修改的文件。

- 返回的 `Document` 中的 `metadata` 字段将显示包含修改文件的 PowerScale 路径。您将使用此路径通过 NFS（或 S3）读取数据，并在应用程序中处理数据（例如：创建块和嵌入）。
- `source` 字段是 PowerScale 上的路径，不一定是您本地系统上的路径（取决于您的挂载策略）； OneFS 将整个存储系统表示为以 `/ifs` 为根的单个数**字**象。
- `change_types` 属性将告知您自上次以来发生了什么变化——例如：新增、修改或删除。

您的 RAG 应用程序可以使用来自 `change_types` 的信息来添加、更新或删除您块和向量存储中的条目。

当使用 `PowerScaleUnstructuredLoader` 时，`page_content` 字段将填充来自非结构化加载器的数据。

## 懒加载

内部来说，PowerScale 和 MetadataIQ 的所有代码都是异步的，加载和懒加载方法将返回一个 python 生成器。我们推荐使用懒加载函数。

In [ ]:
for doc in loader.lazy_load():
    print(doc)  # do something specific with the document

加载函数返回的 `Document` 与上面提到的所有属性都相同。

## 更多示例

您可以在我们的公共 github 网页上找到更多示例和代码：[https://github.com/dell/powerscale-rag-connector/tree/main/examples](https://github.com/dell/powerscale-rag-connector/tree/main/examples)，其中提供了完整的可用示例。

 - [PowerScale LangChain 文件加载器](https://github.com/dell/powerscale-rag-connector/blob/main/examples/powerscale_langchain_doc_loader.py) - 我们标准文件加载器的可用示例
 - [PowerScale LangChain 非结构化加载器](https://github.com/dell/powerscale-rag-connector/blob/main/examples/powerscale_langchain_unstructured_loader.py) - 使用非结构化加载器进行分块和嵌入的标准文件加载器可用示例
 - [PowerScale NVIDIA Retriever 微服务加载器](https://github.com/dell/powerscale-rag-connector/blob/main/examples/powerscale_nvingest_example.py) - 使用 NVIDIA NeMo Retriever 微服务进行分块和嵌入的文件加载器可用示例

## API 参考

有关 PowerScale Document Loader 所有功能和配置的详细文档，请访问 GitHub 页面：[https://github.com/dell/powerscale-rag-connector/](https://github.com/dell/powerscale-rag-connector/)